### Team Data Extraction

In [1]:
# Import packages for data extraction
import pandas as pd
import requests

from selenium  import webdriver
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
# Team data extraction function
def team_data(r):
    x=-1
    
    temp =r.json()
    
    for i in range(0,2):
        
        col_name = []
        row_value = []
        
        col_name.append('match_id')
        row_value.append(temp['match_detail']['match_id'])
        
        col_name.append('home_team_id')
        
        try :
            row_value.append(temp['teams']['home_team_id'])
        except KeyError:
            row_value.append('NA')
        
        col_name.append('home_team_name')
        
        try:
            row_value.append(temp['teams']['home_team_name'])
        except KeyError:
            row_value.append('NA')
        
        
        # id
        name = 'Team_'
        
        for each in ['id','name','short_name']:
            col_name.append(name+each)
            row_value.append(temp['teams']['team'][i][each])
            
        # Points
        name = 'Team_Points_'
        
        for each in ['total','all_out','extras','declare']:
            col_name.append(name+each)
            row_value.append(temp['teams']['team'][i]['stats']['points'][each])
        
        # Raid Points
        
        name = 'Team_Raid_Points_'
        
        for each in ['total','touch','raid_bonus']:
            col_name.append(name+each)
            row_value.append(temp['teams']['team'][i]['stats']['points']['raid_points'][each])
            
        # Tackle Points
        
        name = 'Team_Tackle_Points_'
        
        for each in ['total','capture','capture_bonus']:
            col_name.append(name+each)
            row_value.append(temp['teams']['team'][i]['stats']['points']['tackle_points'][each])
        
        # Raids
        
        name = 'Team_Raids_'
        
        for each in ['total','successful','unsuccessful','Empty']:
            col_name.append(name+each)
            row_value.append(temp['teams']['team'][i]['stats']['raids'][each])
        
        # Tackles
        name = 'Team_Tackles_'
        
        for each in ['total','successful','unsuccessful']:
            col_name.append(name+each)
            row_value.append(temp['teams']['team'][i]['stats']['tackles'][each])
            
        x+=1
        if i ==0:
            team = pd.DataFrame(columns=col_name)
            
        team.loc[x] = row_value
   
    
    return team

In [3]:
# Create team dataframe with 1st match details
url= 'https://www.prokabaddi.com/sifeeds/kabaddi/live/json/1_match.json'
r = requests.get(url)
team  = team_data(r)

In [4]:
# Team dataframe created for 1st match
team

,match_id,home_team_id,home_team_name,Team_id,Team_name,Team_short_name,Team_Points_total,Team_Points_all_out,Team_Points_extras,Team_Points_declare,...,Team_Tackle_Points_total,Team_Tackle_Points_capture,Team_Tackle_Points_capture_bonus,Team_Raids_total,Team_Raids_successful,Team_Raids_unsuccessful,Team_Raids_Empty,Team_Tackles_total,Team_Tackles_successful,Team_Tackles_unsuccessful
0,1,5,U Mumba,3,Jaipur Pink Panthers,JAI,28,2,0,0,...,1,1,0,40,20,10,10,24,1,23
1,1,5,U Mumba,5,U Mumba,MUM,44,4,0,0,...,12,12,0,40,21,4,15,32,10,22


In [5]:
# Shape of Team dataframe
team.shape

(2, 23)

In [6]:
# Extract data for all the available matches
for i in range(2,1801):
    url= f'https://www.prokabaddi.com/sifeeds/kabaddi/live/json/{i}_match.json'
    r = requests.get(url)
    if r.status_code == 200:
        team = pd.concat([team,team_data(r)],sort=False,ignore_index=True)

In [7]:
# shape of team dataframe having all match details
team.shape

(1422, 23)

In [8]:
# Writing to the file.
team.to_excel('team_data.xlsx',index=False)

### Match data Extraction

In [9]:
match = pd.DataFrame(columns=['match_id','match_name','date','start_time','gmtoffset','winning_team',
                             'outcome','value','winning_method','toss_winner','toss_selection','venue'])

In [10]:
match.shape

(0, 12)

In [11]:
count=-1
for i in range(1,1801):
    match_list = []
    url= f'https://www.prokabaddi.com/sifeeds/kabaddi/live/json/{i}_match.json'
    r = requests.get(url)
    if r.status_code == 200:
        r= r.json()
        count+=1
         
        
        match_list.append(r['match_detail']['match_id'])
        match_list.append(r['match_detail']['match_number'])
        match_list.append(r['match_detail']['date'])
        match_list.append(r['match_detail']['start_time'])
        match_list.append(r['match_detail']['gmtoffset'])
        try:
            match_list.append(r['match_detail']['result']['winning_team'])
            match_list.append(r['match_detail']['result']['outcome'])
            match_list.append(r['match_detail']['result']['value'])
            match_list.append(r['match_detail']['result']['winning_method'])
        except TypeError:
            match_list.append('NA')
            match_list.append('NA')
            match_list.append('NA')
            match_list.append('NA')
            
        match_list.append(r['match_detail']['toss']['winner'])
        match_list.append(r['match_detail']['toss']['selection'])
        match_list.append(r['match_detail']['venue']['name'])
        
        match.loc[count]= match_list
     
    i=i+1

In [12]:
match.shape

(711, 12)

In [13]:
match.to_excel('match_data.xlsx',index=False)

### Team Stats Data Extraction

In [14]:
def team_total_points(season,points,point_name):
    driver = webdriver.Firefox()
    driver.get(f'https://www.prokabaddi.com/stats/{season}-{points}-{point_name}-statistics')
    
    try:
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "team_Btn")))
    finally:
        pass
    
    element.click()
    
    while(1):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        try:
            sleep( 1.5 )# time in seconds
            btn = driver.find_element_by_xpath("//*[text()='load more']")
            btn.click()
        except Exception as e:
            break
    
    soup = BeautifulSoup(driver.page_source)
    
    outer_list = []
    
    for each in ['sipk-lb-detailBlock sipk-lb-rank','sipk-lb-playerName',
             'sipk-lb-detailBlock sipk-lb-matchedPlayed','sipk-lb-detailBlock sipk-lb-raidPoints']:
        
        temp = soup.findAll('div',attrs={'class':each})
        
        temp_list=[]
        
        for i in temp:
            temp_list.append(i.find('span').text.replace('.','').strip())
        
        outer_list.append(temp_list)
        
    total_points = pd.DataFrame(outer_list).T
    
    total_points.columns=['Rank','Name','Matches Played','Total Points']
    
    team = soup.findAll('a',attrs={'class':'sipk-lb-detailBlock sipk-lb-team'})
    
    team_list = []
    
    for each in team:
        team_name = each.get('href')
        team_list.append(team_name[7:team_name.find('-profile')])
    
   
    
    driver.quit()
    
    return total_points

In [15]:
team_point  = ['total-points-scored','total-points-conceded','avg-points-scored','successful-raids','raid-points',
        'avg-raid-points','successful-tackles','tackle-points','avg-tackle-points','super-raid',
        'super-tackles','do-or-die-raid-points','all-outs-inflicted','all-outs-conceded']

In [16]:
team_point_code =[96,133,140,13,97,98,15,95,99,134,20,135,136,137]

In [17]:
team_point_dict = dict(zip(team_point,team_point_code))

In [18]:
season = ['All Season','Season 1','Season 2','Season 3','Season 4','Season 5','Season 6','Season 7']

In [19]:
season_code = [0,1,2,3,4,8,26,49]

In [20]:
season_dict = dict(zip(season,season_code))

In [21]:
team_final = pd.DataFrame()

for each_point in team_point:
    for each_season in season:
        temp = team_total_points(season_dict[each_season],team_point_dict[each_point],each_point)
        temp['Season'] = each_season
        temp['Points'] = each_point
        team_final = pd.concat([team_final,temp])

In [22]:
team_final.to_excel('team_stats.xlsx',index=False)